## Course: Computing Foundation of Data Science
## Project Title: Big Data 
## Authors: Kubam Ivo Mbi and Berdai Hasnae


### Task 2.1: Collecting metadata, inspecting schema evolution

In [41]:
import os
# Check wether folder exist if not create
if os.path.exists('Files'):
    pass
else:
    os.mkdir('Files')
    # unzip the files
    # This line of code assumes the zipped file is in the same location as this jupyter notebook
    import zipfile
    
    with zipfile.ZipFile('tlc_0.2perc.zip') as zip_ref:
        zip_ref.extractall('Files/')
#Checking the total number of files
print("Total number of files: ", str(len(os.listdir('Files/tlc_0.2perc'))))

Total number of files:  281


In [36]:
# Create subfolders to hold similiar files
for folder in ['yellow','green','FHV','FHVHV']:
    if os.path.exists(os.path.join('Files', folder)):
        pass
    else:
        os.mkdir(os.path.join('Files', folder))
        
# Reading each file and make a copy in the corresponding subfolder
import shutil

#generator function
def file_content(main_folder):
    file_list = os.listdir(main_folder)
    for file_name in file_list:
        yield file_name


for file in file_content('Files/tlc_0.2perc'):
    for word in ['yellow','green','FHV','FHVHV']:
        if file.lower().startswith('yellow'):
            shutil.copy2(os.path.join('Files/tlc_0.2perc', file),'Files/yellow')
        elif file.lower().startswith('green'):
            shutil.copy2(os.path.join('Files/tlc_0.2perc', file),'Files/green')
        elif file.lower().startswith('fhvhv'):
            shutil.copy2(os.path.join('Files/tlc_0.2perc', file),'Files/FHVHV')
        elif file.lower().startswith('fhv'):
            shutil.copy2(os.path.join('Files/tlc_0.2perc', file),'Files/FHV')


In [49]:
import os           
#Verifying all files were copied
x = 0 # tracking subtotals
y = 0 # cummulative total
for files in ['yellow','green','FHV','FHVHV']:
    x = len(next(os.walk(os.path.join('Files', files)))[2])
    print("Total number of files in ", files," sub-folder is ", str(x))
    y += x
print("Cummulative Total number of files is",  str(y))

Total number of files in  yellow  sub-folder is  131
Total number of files in  green  sub-folder is  76
Total number of files in  FHV  sub-folder is  64
Total number of files in  FHVHV  sub-folder is  10
Cummulative Total number of files is 281
